# Brand Search Lift - part 2


 # <font color="Red">Run the following codes either on dataiku: https://dev-dataiku.walmart.com/projects/DKU_TUTORIAL_BASICS_101_32/recipes/compute_brand_test_control_group_matched/ or Here  
    
    

### In case of large datasets, it is advisable to run it on dataiku due to faster processing. <b>Note: the dataset would need to be changed to your dataset in case of dataiku. The default dataset is: cin_tre

## Variable definition and declaration

In [2]:
# Load Packages
from google.cloud import bigquery
from google.oauth2 import credentials
import pandas as pd
import os
import numpy as np


# Set path to GCP SA credientials file (CHANGE)
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "C:\\Users\\a0g0h5t\\Documents\\WMC\\BQ connection jupyter\\dev_connection_details_1.json"

# Establish connection to BQ (CHANGE)
project = 'wmt-wmg-adops-adhoc-dev'
database = 'cin_tre'


client = bigquery.Client(project=project)

In [3]:
# Mapping Parameters (CHANGE)
brand = 'tineco'
prefix='tineco_v2'
brand_string = '%tineco%'
##start_period = '2022-01-01'
##end_period = '2023-02-24'
# Analysis time frame (this is usually the time frame you are interested in using for the analysis and overlaps with the campaigns)
window_start = '2022-06-02'
window_end = '2022-07-11'
brand_id = '5003561, 571980'
sub_categ_id = '1060215,1060223,18527'
tenant_ste_cd = "'US_GLASS_CORESITE','US_GLASS_IPHONE', 'US_GLASS_ANDROID','US_CORESITE'"
# Parameters to Define Ad Influenced Customers (CHANGE) 
lookback_window = '30' # set to 14 days by default, adjust (lookback_window + media_start_period) if needed
media_start_period = '2022-05-02' # adjusted start period, relative to lookback window


In [4]:
# Inherited Tables
# Features table for test customers as (exposed to sp or display ads)
test_control_no_ads= 'cin_tre.tineco_cust_exposure_search_behavior_purchase_longterm_summary_test_control_noAds_flag'

# Features table for test customers (exposed to display and SP), control (sp only exposed)
test_control_sp_vs_both ='cin_tre.tineco_cust_exposure_search_behavior_purchase_longterm_summary_test_control_sp_vs_both_flag'

# Features table for test customers (exposed to display and SP), control (dsp only exposed)
test_control_dsp_vs_both ='cin_tre.tineco_cust_exposure_search_behavior_purchase_longterm_summary_test_control_dsp_vs_both_flag'


In [5]:
# Define Output Directory (CHANGE)
output_dir = """./outputs/""".format()
if not os.path.exists(output_dir):
    os.mkdir(output_dir)

# Euclidean Distance Matching

## Importing the data

In [6]:
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import NearestNeighbors
pd.set_option('display.max_columns', None)

In [76]:
# Import the features dataset


query = f"""

SELECT
    
 a.*
FROM  `{project}.{test_control_no_ads}` a

"""
print(query)
base_df = client.query(query).to_dataframe()
base_df.head()






SELECT
    
 a.*
FROM  `wmt-wmg-adops-adhoc-dev.cin_tre.tineco_cust_exposure_search_behavior_purchase_longterm_summary_test_control_noAds_flag` a




,pg_cust_id,Test_control_flag,first_exposure_ts,n_branded_search_pre,n_branded_search_post,n_categ_search_pre,n_categ_search_post,tenant_ste_cd,day_of_week,period_of_day,sum_brand_order_pre,sum_brand_revenue_pre,sum_brand_quantity_pre,sum_brand_order_post,sum_brand_revenue_post,sum_brand_quantity_post,sum_categ_order_pre,sum_categ_revenue_pre,sum_categ_quantity_pre,sum_categ_order_post,sum_categ_revenue_post,sum_categ_quantity_post,latest_categ_purchase_longterm,latest_categ_purchase_interval,sum_categ_order_longterm,sum_categ_quantity_longterm,sum_categ_revenue_longterm,latest_brand_purchase_longterm,latest_brand_purchase_interval,sum_brand_order_longterm,sum_brand_quantity_longterm,sum_brand_revenue_longterm
0,00fedd31-5ce4-4404-bc05-8da1312a8918,control,2022-06-22,0,0,3,0,US_GLASS_CORESITE,1,morning,0,0.0,0,0,0.0,0,0,0.0,0,0,0.0,0,NaT,<NA>,1,<NA>,NaN,NaT,<NA>,1,<NA>,NaN
1,05a880c9-3576-4cd0-b938-db126cbd0f36,control,2022-06-22,0,0,1,0,US_GLASS_IPHONE,1,afternoon,0,0.0,0,0,0.0,0,0,0.0,0,0,0.0,0,NaT,<NA>,1,<NA>,NaN,NaT,<NA>,1,<NA>,NaN
2,07256937-319d-4fab-ae3f-5873fac93328,control,2022-06-22,0,0,1,0,US_GLASS_ANDROID,1,afternoon,0,0.0,0,0,0.0,0,0,0.0,0,0,0.0,0,NaT,<NA>,1,<NA>,NaN,NaT,<NA>,1,<NA>,NaN
3,0943ed51-3a38-4453-ae9f-00d7541787a0,control,2022-06-22,0,0,5,0,US_GLASS_ANDROID,1,evening,0,0.0,0,0,0.0,0,0,0.0,0,0,0.0,0,NaT,<NA>,1,<NA>,NaN,NaT,<NA>,1,<NA>,NaN
4,1890db5c-626b-4c60-b292-fdd0104430a6,control,2022-06-22,0,0,1,0,US_GLASS_IPHONE,1,morning,0,0.0,0,0,0.0,0,0,0.0,0,0,0.0,0,NaT,<NA>,1,<NA>,NaN,NaT,<NA>,1,<NA>,NaN


In [77]:
base_df.describe()

,n_branded_search_pre,n_branded_search_post,n_categ_search_pre,n_categ_search_post,day_of_week,sum_brand_order_pre,sum_brand_revenue_pre,sum_brand_quantity_pre,sum_brand_order_post,sum_brand_revenue_post,sum_brand_quantity_post,sum_categ_order_pre,sum_categ_revenue_pre,sum_categ_quantity_pre,sum_categ_order_post,sum_categ_revenue_post,sum_categ_quantity_post,latest_categ_purchase_interval,sum_categ_order_longterm,sum_categ_quantity_longterm,sum_categ_revenue_longterm,latest_brand_purchase_interval,sum_brand_order_longterm,sum_brand_quantity_longterm,sum_brand_revenue_longterm
count,1.048353e+06,1.048353e+06,1.048353e+06,1.048353e+06,1.048353e+06,1.048353e+06,1.048353e+06,1.048353e+06,1.048353e+06,1.048353e+06,1.048353e+06,1.048353e+06,1.048353e+06,1.048353e+06,1.048353e+06,1.048353e+06,1.048353e+06,72391.000000,1.048353e+06,72391.000000,72391.000000,7603.000000,1.048353e+06,7603.000000,7603.000000
mean,1.866261e-02,2.204887e-02,1.188207e+00,1.366878e+00,3.949038e+00,2.297890e-03,3.557290e-01,2.351307e-03,2.206318e-03,3.192613e-01,2.261643e-03,2.186763e-02,2.246905e+00,2.252104e-02,2.699377e-02,2.631383e+00,2.787134e-02,142.938086,1.012563e+00,1.244989,111.186137,131.508615,1.000582e+00,1.103643,151.927668
std,2.155844e-01,2.487441e-01,1.446681e+00,1.743801e+00,2.068461e+00,4.873033e-02,7.871525e+00,5.119080e-02,4.810416e-02,7.278536e+00,5.040621e-02,1.553413e-01,2.357408e+01,1.675383e-01,1.823286e-01,2.587595e+01,2.253173e-01,123.909134,2.546434e-01,2.308480,456.746723,114.425834,2.741017e-02,0.403603,72.307718
min,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,1.000000e+00,0.000000,0.370000,0.000000,1.000000e+00,0.000000,23.440001
25%,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,2.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,15.000000,1.000000e+00,1.000000,32.980000,15.000000,1.000000e+00,1.000000,119.000000
50%,0.000000e+00,0.000000e+00,1.000000e+00,1.000000e+00,4.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,129.000000,1.000000e+00,1.000000,77.320000,117.000000,1.000000e+00,1.000000,129.000000
75%,0.000000e+00,0.000000e+00,2.000000e+00,2.000000e+00,6.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,227.000000,1.000000e+00,1.000000,149.000000,224.000000,1.000000e+00,1.000000,168.000000
max,6.000000e+00,6.000000e+00,6.000000e+00,8.000000e+00,7.000000e+00,3.000000e+00,6.799800e+02,7.000000e+00,5.000000e+00,5.999800e+02,5.000000e+00,3.300000e+01,1.108967e+04,3.300000e+01,3.900000e+01,7.979790e+03,1.160000e+02,435.000000,1.630000e+02,435.000000,100030.149658,433.000000,7.000000e+00,10.000000,2384.000000


In [78]:
len(base_df)

1048353

## Pre-processing<br>
1. Numeric variables: normalization<br>
2. categorical variables: One Hot Encoding <br>

In [79]:
#preprocessing the data
base_df['day_of_week'] = base_df.day_of_week.astype('str')


In [80]:
#base_df.columns.values
#Selecting all the required columns for analysis
req_columns = [ 'Test_control_flag', 
                'pg_cust_id',
                'n_branded_search_pre',  
                'n_categ_search_pre',
                'tenant_ste_cd',
                'day_of_week',
                'period_of_day',
                
                'sum_brand_order_pre',
                'sum_brand_revenue_pre',
                'sum_brand_quantity_pre',
                'sum_categ_order_pre',
                'sum_categ_revenue_pre',
                'sum_categ_quantity_pre',
                'latest_categ_purchase_interval',
                'sum_categ_order_longterm', 
                'sum_categ_quantity_longterm',
                'sum_categ_revenue_longterm', 
                'latest_brand_purchase_interval',
                'sum_brand_order_longterm',
                'sum_brand_quantity_longterm',
                'sum_brand_revenue_longterm']

#'pre_period_ad_exposure',

In [81]:
test_group = base_df.loc[base_df.Test_control_flag=='test',req_columns]
test_group.head()

,Test_control_flag,pg_cust_id,n_branded_search_pre,n_categ_search_pre,tenant_ste_cd,day_of_week,period_of_day,sum_brand_order_pre,sum_brand_revenue_pre,sum_brand_quantity_pre,sum_categ_order_pre,sum_categ_revenue_pre,sum_categ_quantity_pre,latest_categ_purchase_interval,sum_categ_order_longterm,sum_categ_quantity_longterm,sum_categ_revenue_longterm,latest_brand_purchase_interval,sum_brand_order_longterm,sum_brand_quantity_longterm,sum_brand_revenue_longterm
7,test,2539974a-38da-437c-937e-3f6e6fadadd2,0,1,US_GLASS_CORESITE,1,morning,0,0.0,0,0,0.0,0,<NA>,1,<NA>,NaN,<NA>,1,<NA>,NaN
15,test,48e3ca47-e2b7-4cee-acc0-7b204d262cb3,0,1,US_GLASS_IPHONE,1,evening,0,0.0,0,0,0.0,0,<NA>,1,<NA>,NaN,<NA>,1,<NA>,NaN
25,test,80ae720e-d879-41a5-a87f-d22befb34326,0,1,US_GLASS_IPHONE,1,morning,0,0.0,0,0,0.0,0,<NA>,1,<NA>,NaN,<NA>,1,<NA>,NaN
28,test,d528497c-7c0a-4f94-9f13-29618ca92282,0,1,US_GLASS_ANDROID,1,afternoon,0,0.0,0,0,0.0,0,<NA>,1,<NA>,NaN,<NA>,1,<NA>,NaN
29,test,d5f5cb7f-7659-4b18-98aa-af3aef9d4bda,0,1,US_GLASS_ANDROID,1,afternoon,0,0.0,0,0,0.0,0,<NA>,1,<NA>,NaN,<NA>,1,<NA>,NaN


In [82]:
len(test_group)

247946

In [83]:
control_group = base_df.loc[base_df.Test_control_flag=='control',req_columns]
control_group.head()

,Test_control_flag,pg_cust_id,n_branded_search_pre,n_categ_search_pre,tenant_ste_cd,day_of_week,period_of_day,sum_brand_order_pre,sum_brand_revenue_pre,sum_brand_quantity_pre,sum_categ_order_pre,sum_categ_revenue_pre,sum_categ_quantity_pre,latest_categ_purchase_interval,sum_categ_order_longterm,sum_categ_quantity_longterm,sum_categ_revenue_longterm,latest_brand_purchase_interval,sum_brand_order_longterm,sum_brand_quantity_longterm,sum_brand_revenue_longterm
0,control,00fedd31-5ce4-4404-bc05-8da1312a8918,0,3,US_GLASS_CORESITE,1,morning,0,0.0,0,0,0.0,0,<NA>,1,<NA>,NaN,<NA>,1,<NA>,NaN
1,control,05a880c9-3576-4cd0-b938-db126cbd0f36,0,1,US_GLASS_IPHONE,1,afternoon,0,0.0,0,0,0.0,0,<NA>,1,<NA>,NaN,<NA>,1,<NA>,NaN
2,control,07256937-319d-4fab-ae3f-5873fac93328,0,1,US_GLASS_ANDROID,1,afternoon,0,0.0,0,0,0.0,0,<NA>,1,<NA>,NaN,<NA>,1,<NA>,NaN
3,control,0943ed51-3a38-4453-ae9f-00d7541787a0,0,5,US_GLASS_ANDROID,1,evening,0,0.0,0,0,0.0,0,<NA>,1,<NA>,NaN,<NA>,1,<NA>,NaN
4,control,1890db5c-626b-4c60-b292-fdd0104430a6,0,1,US_GLASS_IPHONE,1,morning,0,0.0,0,0,0.0,0,<NA>,1,<NA>,NaN,<NA>,1,<NA>,NaN


In [84]:
  # Replace the NaN values in the test group and control group with 0 and 9999.
#test_group = test_group.fillna(-1)
#control_group = control_group.fillna(-1)

In [85]:
test_group = test_group.drop('Test_control_flag', axis=1)
control_group = control_group.drop('Test_control_flag', axis=1)

In [86]:
def scaling_dataframes(test_group):
    
    sc = StandardScaler()

    test_group_copy= test_group.loc[:,test_group.columns !='pg_cust_id'].copy()
      # Replace the NaN values in the test group and control group with 0 and 9999.
    test_group_copy[['latest_categ_purchase_interval','latest_brand_purchase_interval']] = test_group_copy[['latest_categ_purchase_interval','latest_brand_purchase_interval']].fillna(9999)
    test_group_copy = test_group_copy.fillna(0)

    # get numeric data
    num_d = test_group_copy.select_dtypes(exclude=['object'])

     # update the cols with their normalized values
    test_group_copy[num_d.columns] = sc.fit_transform(num_d)

    # convert categorical/string variable to One Hot Encoding
    test_group_copy = pd.get_dummies(test_group_copy)
    return test_group_copy

In [87]:
test_group_copy = scaling_dataframes(test_group)
test_group_copy.head()

,n_branded_search_pre,n_categ_search_pre,sum_brand_order_pre,sum_brand_revenue_pre,sum_brand_quantity_pre,sum_categ_order_pre,sum_categ_revenue_pre,sum_categ_quantity_pre,latest_categ_purchase_interval,sum_categ_order_longterm,sum_categ_quantity_longterm,sum_categ_revenue_longterm,latest_brand_purchase_interval,sum_brand_order_longterm,sum_brand_quantity_longterm,sum_brand_revenue_longterm,tenant_ste_cd_US_GLASS_ANDROID,tenant_ste_cd_US_GLASS_CORESITE,tenant_ste_cd_US_GLASS_IPHONE,day_of_week_1,day_of_week_2,day_of_week_3,day_of_week_4,day_of_week_5,day_of_week_6,day_of_week_7,period_of_day_afternoon,period_of_day_evening,period_of_day_morning
7,-0.112592,-0.237323,-0.063033,-0.059853,-0.061653,-0.189563,-0.13827,-0.180136,0.327714,-0.092471,-0.204618,-0.1821,0.106265,-0.027676,-0.098268,-0.093156,0,1,0,1,0,0,0,0,0,0,0,0,1
15,-0.112592,-0.237323,-0.063033,-0.059853,-0.061653,-0.189563,-0.13827,-0.180136,0.327714,-0.092471,-0.204618,-0.1821,0.106265,-0.027676,-0.098268,-0.093156,0,0,1,1,0,0,0,0,0,0,0,1,0
25,-0.112592,-0.237323,-0.063033,-0.059853,-0.061653,-0.189563,-0.13827,-0.180136,0.327714,-0.092471,-0.204618,-0.1821,0.106265,-0.027676,-0.098268,-0.093156,0,0,1,1,0,0,0,0,0,0,0,0,1
28,-0.112592,-0.237323,-0.063033,-0.059853,-0.061653,-0.189563,-0.13827,-0.180136,0.327714,-0.092471,-0.204618,-0.1821,0.106265,-0.027676,-0.098268,-0.093156,1,0,0,1,0,0,0,0,0,0,1,0,0
29,-0.112592,-0.237323,-0.063033,-0.059853,-0.061653,-0.189563,-0.13827,-0.180136,0.327714,-0.092471,-0.204618,-0.1821,0.106265,-0.027676,-0.098268,-0.093156,1,0,0,1,0,0,0,0,0,0,1,0,0


In [19]:
control_group_copy = scaling_dataframes(control_group)
control_group_copy.head()

,n_branded_search_pre,n_categ_search_pre,sum_brand_order_pre,sum_brand_revenue_pre,sum_brand_quantity_pre,sum_categ_order_pre,sum_categ_revenue_pre,sum_categ_quantity_pre,latest_categ_purchase_interval,sum_categ_order_longterm,sum_categ_quantity_longterm,sum_categ_revenue_longterm,latest_brand_purchase_interval,sum_brand_order_longterm,sum_brand_quantity_longterm,sum_brand_revenue_longterm,tenant_ste_cd_US_GLASS_ANDROID,tenant_ste_cd_US_GLASS_CORESITE,tenant_ste_cd_US_GLASS_IPHONE,day_of_week_1,day_of_week_2,day_of_week_3,day_of_week_4,day_of_week_5,day_of_week_6,day_of_week_7,period_of_day_afternoon,period_of_day_evening,period_of_day_morning
0,-0.077351,1.271341,-0.041023,-0.039582,-0.039839,-0.122376,-0.079815,-0.117199,0.25351,-0.039667,-0.105774,-0.04907,0.077944,-0.018794,-0.073854,-0.071592,0,1,0,1,0,0,0,0,0,0,0,0,1
1,-0.077351,-0.098667,-0.041023,-0.039582,-0.039839,-0.122376,-0.079815,-0.117199,0.25351,-0.039667,-0.105774,-0.04907,0.077944,-0.018794,-0.073854,-0.071592,0,0,1,1,0,0,0,0,0,0,1,0,0
2,-0.077351,-0.098667,-0.041023,-0.039582,-0.039839,-0.122376,-0.079815,-0.117199,0.25351,-0.039667,-0.105774,-0.04907,0.077944,-0.018794,-0.073854,-0.071592,1,0,0,1,0,0,0,0,0,0,1,0,0
3,-0.077351,2.641349,-0.041023,-0.039582,-0.039839,-0.122376,-0.079815,-0.117199,0.25351,-0.039667,-0.105774,-0.04907,0.077944,-0.018794,-0.073854,-0.071592,1,0,0,1,0,0,0,0,0,0,0,1,0
4,-0.077351,-0.098667,-0.041023,-0.039582,-0.039839,-0.122376,-0.079815,-0.117199,0.25351,-0.039667,-0.105774,-0.04907,0.077944,-0.018794,-0.073854,-0.071592,0,0,1,1,0,0,0,0,0,0,0,0,1


## Implement Matching

In [20]:
def get_matching_pairs(test_group_copy,control_group, control_group_copy):

    nbrs = NearestNeighbors(n_neighbors=1, algorithm='ball_tree').fit(control_group_copy)
    distances, indices = nbrs.kneighbors(test_group_copy)
    indices = indices.reshape(indices.shape[0])
    matched = control_group.iloc[indices]
    return matched

In [21]:
matched_control= get_matching_pairs(test_group_copy,control_group, control_group_copy)

In [22]:
matched_control.head()

,pg_cust_id,n_branded_search_pre,n_categ_search_pre,tenant_ste_cd,day_of_week,period_of_day,sum_brand_order_pre,sum_brand_revenue_pre,sum_brand_quantity_pre,sum_categ_order_pre,sum_categ_revenue_pre,sum_categ_quantity_pre,latest_categ_purchase_interval,sum_categ_order_longterm,sum_categ_quantity_longterm,sum_categ_revenue_longterm,latest_brand_purchase_interval,sum_brand_order_longterm,sum_brand_quantity_longterm,sum_brand_revenue_longterm
131,d505c13c-8ee3-4af5-a640-397085ec36a5,0,1,US_GLASS_CORESITE,1,morning,0,0.0,0,0,0.0,0,<NA>,1,<NA>,NaN,<NA>,1,<NA>,NaN
18,59f1f625-9993-4d2c-adb9-7fd8769aba09,0,1,US_GLASS_IPHONE,1,evening,0,0.0,0,0,0.0,0,<NA>,1,<NA>,NaN,<NA>,1,<NA>,NaN
30,dec9aa25-0da5-4229-84ca-639637d43599,0,1,US_GLASS_IPHONE,1,morning,0,0.0,0,0,0.0,0,<NA>,1,<NA>,NaN,<NA>,1,<NA>,NaN
2,07256937-319d-4fab-ae3f-5873fac93328,0,1,US_GLASS_ANDROID,1,afternoon,0,0.0,0,0,0.0,0,<NA>,1,<NA>,NaN,<NA>,1,<NA>,NaN
2,07256937-319d-4fab-ae3f-5873fac93328,0,1,US_GLASS_ANDROID,1,afternoon,0,0.0,0,0,0.0,0,<NA>,1,<NA>,NaN,<NA>,1,<NA>,NaN


In [35]:
len(test_group_copy)

247946

## Merge the matched control data with the original base table to extract features for the matched dataset

In [32]:
matched_control_full_data = pd.merge(base_df,matched_control[['pg_cust_id']],on='pg_cust_id',how='right')

In [40]:
matched_control_full_data.head()

,pg_cust_id,Test_control_flag,first_exposure_ts,n_branded_search_pre,n_branded_search_post,n_categ_search_pre,n_categ_search_post,tenant_ste_cd,day_of_week,period_of_day,sum_brand_order_pre,sum_brand_revenue_pre,sum_brand_quantity_pre,sum_brand_order_post,sum_brand_revenue_post,sum_brand_quantity_post,sum_categ_order_pre,sum_categ_revenue_pre,sum_categ_quantity_pre,sum_categ_order_post,sum_categ_revenue_post,sum_categ_quantity_post,latest_categ_purchase_longterm,latest_categ_purchase_interval,sum_categ_order_longterm,sum_categ_quantity_longterm,sum_categ_revenue_longterm,latest_brand_purchase_longterm,latest_brand_purchase_interval,sum_brand_order_longterm,sum_brand_quantity_longterm,sum_brand_revenue_longterm
0,d505c13c-8ee3-4af5-a640-397085ec36a5,control,2022-06-22,0,0,1,0,US_GLASS_CORESITE,1,morning,0,0.0,0,0,0.0,0,0,0.0,0,0,0.0,0,NaT,<NA>,1,<NA>,NaN,NaT,<NA>,1,<NA>,NaN
1,59f1f625-9993-4d2c-adb9-7fd8769aba09,control,2022-06-22,0,0,1,0,US_GLASS_IPHONE,1,evening,0,0.0,0,0,0.0,0,0,0.0,0,0,0.0,0,NaT,<NA>,1,<NA>,NaN,NaT,<NA>,1,<NA>,NaN
2,dec9aa25-0da5-4229-84ca-639637d43599,control,2022-06-22,0,0,1,0,US_GLASS_IPHONE,1,morning,0,0.0,0,0,0.0,0,0,0.0,0,0,0.0,0,NaT,<NA>,1,<NA>,NaN,NaT,<NA>,1,<NA>,NaN
3,07256937-319d-4fab-ae3f-5873fac93328,control,2022-06-22,0,0,1,0,US_GLASS_ANDROID,1,afternoon,0,0.0,0,0,0.0,0,0,0.0,0,0,0.0,0,NaT,<NA>,1,<NA>,NaN,NaT,<NA>,1,<NA>,NaN
4,07256937-319d-4fab-ae3f-5873fac93328,control,2022-06-22,0,0,1,0,US_GLASS_ANDROID,1,afternoon,0,0.0,0,0,0.0,0,0,0.0,0,0,0.0,0,NaT,<NA>,1,<NA>,NaN,NaT,<NA>,1,<NA>,NaN


In [41]:
matched_control_full_data.dtypes

pg_cust_id                                object
Test_control_flag                         object
first_exposure_ts                 datetime64[ns]
n_branded_search_pre                       Int64
n_branded_search_post                      Int64
n_categ_search_pre                         Int64
n_categ_search_post                        Int64
tenant_ste_cd                             object
day_of_week                               object
period_of_day                             object
sum_brand_order_pre                        Int64
sum_brand_revenue_pre                    float64
sum_brand_quantity_pre                     Int64
sum_brand_order_post                       Int64
sum_brand_revenue_post                   float64
sum_brand_quantity_post                    Int64
sum_categ_order_pre                        Int64
sum_categ_revenue_pre                    float64
sum_categ_quantity_pre                     Int64
sum_categ_order_post                       Int64
sum_categ_revenue_po

In [45]:
#create schema, job_config, and table in BigQuery
#the df in load_table_from_dateframe is called "merged"


job_config = bigquery.LoadJobConfig(schema=[bigquery.SchemaField("pg_cust_id", bigquery.enums.SqlTypeNames.STRING),
                                            bigquery.SchemaField("Test_control_flag", bigquery.enums.SqlTypeNames.STRING),
                                            bigquery.SchemaField("first_exposure_ts", bigquery.enums.SqlTypeNames.DATETIME),
                                            bigquery.SchemaField("n_branded_search_pre", bigquery.enums.SqlTypeNames.INT64),
                                            bigquery.SchemaField("n_branded_search_post", bigquery.enums.SqlTypeNames.INT64),
                                            bigquery.SchemaField("n_categ_search_pre", bigquery.enums.SqlTypeNames.INT64),
                                            bigquery.SchemaField("n_categ_search_post", bigquery.enums.SqlTypeNames.INT64),
                                           bigquery.SchemaField("tenant_ste_cd", bigquery.enums.SqlTypeNames.STRING),
                                            bigquery.SchemaField("day_of_week", bigquery.enums.SqlTypeNames.STRING),
                                            bigquery.SchemaField("period_of_day", bigquery.enums.SqlTypeNames.STRING),
                                            bigquery.SchemaField("sum_brand_order_pre", bigquery.enums.SqlTypeNames.INT64),
                                            bigquery.SchemaField("sum_brand_revenue_pre", bigquery.enums.SqlTypeNames.FLOAT64),
                                           bigquery.SchemaField("sum_brand_quantity_pre", bigquery.enums.SqlTypeNames.INT64),
                                           bigquery.SchemaField("sum_brand_order_post", bigquery.enums.SqlTypeNames.INT64),
                                            bigquery.SchemaField("sum_brand_revenue_post", bigquery.enums.SqlTypeNames.FLOAT64),
                                           bigquery.SchemaField("sum_brand_quantity_post", bigquery.enums.SqlTypeNames.INT64),
                                           bigquery.SchemaField("sum_categ_order_pre", bigquery.enums.SqlTypeNames.INT64),
                                            bigquery.SchemaField("sum_categ_revenue_pre", bigquery.enums.SqlTypeNames.FLOAT64),
                                           bigquery.SchemaField("sum_categ_quantity_pre", bigquery.enums.SqlTypeNames.INT64),
                                           bigquery.SchemaField("sum_categ_order_post", bigquery.enums.SqlTypeNames.INT64),
                                            bigquery.SchemaField("sum_categ_revenue_post", bigquery.enums.SqlTypeNames.FLOAT64),
                                           bigquery.SchemaField("sum_categ_quantity_post", bigquery.enums.SqlTypeNames.INT64),
                                            bigquery.SchemaField("latest_categ_purchase_longterm", bigquery.enums.SqlTypeNames.DATE),
                                            bigquery.SchemaField("latest_categ_purchase_interval", bigquery.enums.SqlTypeNames.INT64),
                                            bigquery.SchemaField("sum_categ_order_longterm", bigquery.enums.SqlTypeNames.FLOAT64),
                                            bigquery.SchemaField("sum_categ_quantity_longterm", bigquery.enums.SqlTypeNames.FLOAT64),
                                            bigquery.SchemaField("sum_categ_revenue_longterm", bigquery.enums.SqlTypeNames.FLOAT64),
                                            bigquery.SchemaField("latest_brand_purchase_longterm", bigquery.enums.SqlTypeNames.DATE),
                                            bigquery.SchemaField("latest_brand_purchase_interval", bigquery.enums.SqlTypeNames.INT64),
                                            bigquery.SchemaField("sum_brand_order_longterm", bigquery.enums.SqlTypeNames.FLOAT64),
                                            bigquery.SchemaField("sum_brand_quantity_longterm", bigquery.enums.SqlTypeNames.FLOAT64),
                                            bigquery.SchemaField("sum_brand_revenue_longterm", bigquery.enums.SqlTypeNames.FLOAT64)
                                           ],
                                    write_disposition="WRITE_TRUNCATE",)
job = client.load_table_from_dataframe(
    matched_control_full_data, f"{project}.{database}.{prefix}_control_group_matched", job_config=job_config
    )

## Combine the test and control data into 1 table

In [50]:
# Union test and control


query = f"""
CREATE OR REPLACE TABLE `{project}.{database}.{prefix}_brand_test_control_group_matched`
as

SELECT    
     pg_cust_id,
     Test_control_flag,
     CAST(first_exposure_ts AS TIMESTAMP) AS FIRST_EXPOSURE_TS,
    n_branded_search_pre,
    n_branded_search_post,
    n_categ_search_pre,
    n_categ_search_post,
    tenant_ste_cd,
    CAST(day_of_week AS STRING) AS DAY_OF_WEEK,
    period_of_day,
    sum_brand_order_pre,
    sum_brand_revenue_pre,
    sum_brand_quantity_pre,
    sum_brand_order_post,
    sum_brand_revenue_post,
    sum_brand_quantity_post,
    sum_categ_order_pre,
    sum_categ_revenue_pre,
    sum_categ_quantity_pre,
    sum_categ_order_post,
    sum_categ_revenue_post,
    sum_categ_quantity_post,
    latest_categ_purchase_longterm,
    latest_categ_purchase_interval,
    sum_categ_order_longterm,
    sum_categ_quantity_longterm,
    sum_categ_revenue_longterm,
    latest_brand_purchase_longterm,
    latest_brand_purchase_interval,
    sum_brand_order_longterm,
    sum_brand_quantity_longterm,
    sum_brand_revenue_longterm
FROM  `{project}.{test_control_no_ads}` a
where Test_control_flag='test'

UNION ALL

SELECT 
     pg_cust_id,
     Test_control_flag,
     CAST(first_exposure_ts AS TIMESTAMP) AS FIRST_EXPOSURE_TS,
    n_branded_search_pre,
    n_branded_search_post,
    n_categ_search_pre,
    n_categ_search_post,
    tenant_ste_cd,
    CAST(day_of_week AS STRING) AS DAY_OF_WEEK,
    period_of_day,
    sum_brand_order_pre,
    sum_brand_revenue_pre,
    sum_brand_quantity_pre,
    sum_brand_order_post,
    sum_brand_revenue_post,
    sum_brand_quantity_post,
    sum_categ_order_pre,
    sum_categ_revenue_pre,
    sum_categ_quantity_pre,
    sum_categ_order_post,
    sum_categ_revenue_post,
    sum_categ_quantity_post,
    latest_categ_purchase_longterm,
    latest_categ_purchase_interval,
    sum_categ_order_longterm,
    sum_categ_quantity_longterm,
    sum_categ_revenue_longterm,
    latest_brand_purchase_longterm,
    latest_brand_purchase_interval,
    sum_brand_order_longterm,
    sum_brand_quantity_longterm,
    sum_brand_revenue_longterm
FROM `{project}.{database}.{prefix}_control_group_matched`

"""
print(query)
QUERY_JOB = client.query(query).to_dataframe()
QUERY_JOB.head()



CREATE OR REPLACE TABLE `wmt-wmg-adops-adhoc-dev.cin_tre.tineco_v2_brand_test_control_group_matched`
as

SELECT    
     pg_cust_id,
     Test_control_flag,
     CAST(first_exposure_ts AS TIMESTAMP) AS FIRST_EXPOSURE_TS,
    n_branded_search_pre,
    n_branded_search_post,
    n_categ_search_pre,
    n_categ_search_post,
    tenant_ste_cd,
    CAST(day_of_week AS STRING) AS DAY_OF_WEEK,
    period_of_day,
    sum_brand_order_pre,
    sum_brand_revenue_pre,
    sum_brand_quantity_pre,
    sum_brand_order_post,
    sum_brand_revenue_post,
    sum_brand_quantity_post,
    sum_categ_order_pre,
    sum_categ_revenue_pre,
    sum_categ_quantity_pre,
    sum_categ_order_post,
    sum_categ_revenue_post,
    sum_categ_quantity_post,
    latest_categ_purchase_longterm,
    latest_categ_purchase_interval,
    sum_categ_order_longterm,
    sum_categ_quantity_longterm,
    sum_categ_revenue_longterm,
    latest_brand_purchase_longterm,
    latest_brand_purchase_interval,
    sum_brand_order_longt

""


In [51]:
#QC
query = f"""

SELECT
    
 TEST_CONTROL_FLAG,COUNT(DISTINCT PG_CUST_ID) AS PG_CUST_IDS
FROM  `{project}.{database}.{prefix}_brand_test_control_group_matched` a
GROUP BY 1
"""
print(query)
base_df = client.query(query).to_dataframe()
base_df.head()



SELECT
    
 TEST_CONTROL_FLAG,COUNT(DISTINCT PG_CUST_ID) AS PG_CUST_IDS
FROM  `wmt-wmg-adops-adhoc-dev.cin_tre.tineco_v2_brand_test_control_group_matched` a
GROUP BY 1



,TEST_CONTROL_FLAG,PG_CUST_IDS
0,test,247946
1,control,8704


*<b> The control group data is matched with replacement which means that it is possible for a control group customer to have more than 1 matching test group*

In [55]:
# QC: comparison after matching
query = f"""
select Test_control_flag,
count(distinct pg_cust_id) as pg_cust_id,
sum(n_branded_search_pre)/count( pg_cust_id) as avg_brand_searches_pre,
sum(n_categ_search_pre)/count( pg_cust_id) as avg_categ_searches_pre,
avg(latest_brand_purchase_interval) as avg_days_between_exposure_purch_brand,
sum(sum_brand_revenue_pre)/count( case when sum_brand_revenue_pre >0 then pg_cust_id else NULL end) as avg_revenue_brand_pre,
sum(sum_categ_revenue_pre)/count( case when sum_categ_revenue_pre >0 then pg_cust_id else NULL end) as avg_categ_revenue_pre,
sum(sum_brand_revenue_pre) as total_brand_rev_pre,
sum(n_categ_search_pre) as total_categ_search_pre
 from

`{project}.{database}.{prefix}_brand_test_control_group_matched`
group by 1

"""
print(query)
df = client.query(query).to_dataframe()
df.head()



select Test_control_flag,
count(distinct pg_cust_id) as pg_cust_id,
sum(n_branded_search_pre)/count( pg_cust_id) as avg_brand_searches_pre,
sum(n_categ_search_pre)/count( pg_cust_id) as avg_categ_searches_pre,
avg(latest_brand_purchase_interval) as avg_days_between_exposure_purch_brand,
sum(sum_brand_revenue_pre)/count( case when sum_brand_revenue_pre >0 then pg_cust_id else NULL end) as avg_revenue_brand_pre,
sum(sum_categ_revenue_pre)/count( case when sum_categ_revenue_pre >0 then pg_cust_id else NULL end) as avg_categ_revenue_pre,
sum(sum_brand_revenue_pre) as total_brand_rev_pre,
sum(n_categ_search_pre) as total_categ_search_pre
 from

`wmt-wmg-adops-adhoc-dev.cin_tre.tineco_v2_brand_test_control_group_matched`
group by 1




,Test_control_flag,pg_cust_id,avg_brand_searches_pre,avg_categ_searches_pre,avg_days_between_exposure_purch_brand,avg_revenue_brand_pre,avg_categ_revenue_pre,total_brand_rev_pre,total_categ_search_pre
0,test,247946,0.027583,1.330786,122.517515,152.146162,104.017164,151841.869896,329963
1,control,8704,0.026022,1.328596,173.925217,111.467595,86.193584,111244.659546,329420
